S12 T01: Pipelines, grid search i text mining

# Statement
Let's start getting familiar with Pipelines, grid search and text mining !!!! Let's start with some basic exercises

Level 1

- Exercise 1

Take the dataset you prefer and perform a pipeline and a gridsearch applying the Random Forest algorithm.

- Exercise 2

Take any English text you choose, and calculate the frequency of the words.

Level 2

- Exercise 1

Find the stopwords and perform stemming on your dataset.

Level 3

- Exercise 1

Perform sentiment analysis on your set of words.

Resources
Classroom resources and <https://www.nltk.org>

# Preprocessing

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [3]:
# settings to display all columns (default is 20, now is None (all))
pd.set_option("display.max_columns", None)

In [4]:
# Import cleaned and sampled train an test dataset from previous Task.
df_train = pd.read_csv('..\data\DelayedFlights_train.csv')
df_test  = pd.read_csv('..\data\DelayedFlights_test.csv')

### Explanation of the Train / Test Sample 
* Is imported from previous Task (S09T01).  
* Is 1% of the original dataset, randomly sampled and stratified by Airline.
* Is parted 33% test and 66% train.

In [5]:
# Let's delete the first column
df_train = df_train.drop(columns='Unnamed: 0')
df_test  = df_test.drop(columns='Unnamed: 0')

### Deletion of DepDelay attribute 


In [6]:
# Let's delete the first column
df_train = df_train.drop(columns='DepDelay')
df_test  = df_test.drop(columns='DepDelay')

### Concatenation Train / Test sets.

In [7]:
#Let's concatenate df_train & df_test
df_complete = pd.concat([df_train,df_test])
df_complete.shape

(19283, 33)

In [8]:
# Our Y or Target is ArrDelay:
y = df_complete.ArrDelay
y = y.array # Convert pandas series to numpy array
type(y)

pandas.core.arrays.numpy_.PandasArray

In [9]:
# Our X now is going to be all the columns in df_complete except ArrDelay and Date (also the OHE of Airline)
X = df_complete.drop(columns=["ArrDelay","Date"])
feature_list = list(X.columns) # Saving feature names for later use
X = X.to_numpy() # Convert dataframe to array
type(X)

numpy.ndarray

# Level 1

## - Exercise 1 - Random Forest Regression

Take the dataset you prefer and perform a pipeline and a gridsearch applying the Random Forest algorithm.

We are going to use the Delayed Flights dataset, without feature DepDelay.

First, let's calculate the Accuracy of Random Forest model with default parameters.

In [10]:
# Instantiate model with 100 decision trees
model3 = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
model3.fit(X, y)
# Calculate R2 to see the accuracy of the model with all data.
r_sq_train3 = model3.score(X,y)
print('coefficient of determination with all data: %.3f' %r_sq_train3)

coefficient of determination with all data: 0.993


In [11]:
# Applying k-Fold Cross Validation (CV) with all data
accuracies3 = cross_val_score(estimator = model3, X=X, y=y , cv = 10) # default
print("Random Forest Regression:\n Accuracy with train data: %.3f"%accuracies3.mean(), "+/- %3.f"%accuracies3.std(),"\n")

Random Forest Regression:
 Accuracy with train data: 0.948 +/-   0 



The Random Forest Model, with n_estimators = 100 (default), has an accuracy of 0.948 (quite good)  

Let's try to increase the Cross Validation Accuracy by tuning parameters with GridSearch and Pipeline.

Definition of <pipeline> class according to scikit-learn is:   
>*Sequentially apply a list of transforms and a final estimator.*  
>*Intermediate steps of pipeline must implement fit and transform methods and the final estimator only needs to implement fit.*    

### Observation 3.
* Now we can really know if the random forest regression model is good or not!
* And the answer is that is very good! With test data, the $R^2$ is 0.927 and the accuracy of the Cross Validation is quite good (0.865 +/- 0.059).
* The explanation now is that, even without the DepDelay variable, the random forest model can predict test data with good accuracy.


## - Exercise 2

Take any English text you choose, and calculate the frequency of the words.



# Level 2 

## - Exercise 1

Find the stopwords and perform stemming on your dataset.



# Level 3

## - Exercise 1

Perform sentiment analysis on your set of words.